In [2]:
import sys
sys.path.append("../scripts")

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

import tad_data

In [50]:
position_input = keras.Input(shape=(2048, 25, 3))
pos = layers.Conv2D(32, 3, activation="leaky_relu", padding="same")(position_input)
pos = layers.MaxPool2D(2, 2, padding="same")(pos)

pos = layers.Conv2D(64, 3, activation="leaky_relu", padding="same")(pos)
pos = layers.MaxPool2D(2, 2, padding="same")(pos)

motion_input = keras.Input(shape=(2048, 25, 3))
mt = layers.Conv2D(32, 3, activation="leaky_relu", padding="same")(motion_input)
mt = layers.MaxPool2D(2, 2, padding="same")(mt)

mt = layers.Conv2D(64, 3, activation="leaky_relu", padding="same")(mt)
mt = layers.MaxPool2D(2, 2, padding="same")(mt)

x = layers.Concatenate()([pos, mt])

x = layers.Conv2D(64, 1, activation="leaky_relu", padding="same")(x)

x = layers.Conv2D(128, 3, activation="leaky_relu", padding="same")(x)
x = layers.MaxPool2D(2, 2, padding="same")(x)

x = layers.Conv2D(256, 3, activation="leaky_relu", padding="same")(x)
x = layers.MaxPool2D(2, 2, padding="same")(x)

x = layers.AveragePooling2D((1, 2))(x)

x = layers.Conv2DTranspose(128, (4, 1), strides=(2, 1), activation="leaky_relu", padding="same")(x)
x = layers.Conv2DTranspose(128, (4, 1), strides=(2, 1), activation="leaky_relu", padding="same")(x)
x = layers.Conv2DTranspose(128, (4, 1), strides=(2, 1), activation="leaky_relu", padding="same")(x)

heatmap = layers.Conv2D(10, (3, 1), activation="leaky_relu", padding="same")(x)
heatmap = layers.Conv2D(10, (1, 1), activation="sigmoid", padding="same")(heatmap)

offset = layers.Conv2D(1, (3, 1), activation="leaky_relu", padding="same")(x)
offset = layers.Conv2D(1, (1, 1), activation=None, padding="same")(offset)

size = layers.Conv2D(1, (3, 1), activation="leaky_relu", padding="same")(x)
size = layers.Conv2D(1, (1, 1), activation=None, padding="same")(size)

model = keras.Model([position_input, motion_input], [heatmap, offset, size])
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 2048, 25,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 2048, 25,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 2048, 25,  │        896 │ input_layer_12[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 2048, 25,  │        896 │ input_layer_13[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_36    │ (None, 1024, 13,  │          0 │ conv2d_78[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_38    │ (None, 1024, 13,  │          0 │ conv2d_80[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 1024, 13,  │     18,496 │ max_pooling2d_36… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 1024, 13,  │     18,496 │ max_pooling2d_38… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_37    │ (None, 512, 7,    │          0 │ conv2d_79[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_39    │ (None, 512, 7,    │          0 │ conv2d_81[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 512, 7,    │          0 │ max_pooling2d_37… │
│ (Concatenate)       │ 128)              │            │ max_pooling2d_39… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_82 (Conv2D)  │ (None, 512, 7,    │      8,256 │ concatenate_6[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_83 (Conv2D)  │ (None, 512, 7,    │     73,856 │ conv2d_82[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_40    │ (None, 256, 4,    │          0 │ conv2d_83[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_84 (Conv2D)  │ (None, 256, 4,    │    295,168 │ max_pooling2d_40… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_41    │ (None, 128, 2,    │          0 │ conv2d_84[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_6 │ (None, 128, 1,    │          0 │ max_pooling2d_41

 Total params: 683,326 (2.61 MB)

 Trainable params: 683,326 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
class HeatMapCrossEntropy(losses.Loss):
    def __init__(self, alpha=2, beta=4, reduce=True, name="heatmap_ce"):
        super().__init__(name=name, reduction=None)
        self.alpha = alpha
        self.beta = beta
        self.reduce = reduce

    def call(self, target, output):
        output = tf.clip_by_value(output, keras.backend.epsilon(), 1. - keras.backend.epsilon())
        
        mask = tf.math.floor(target)
        
        hce = mask * tf.math.pow((1 - output), self.alpha) * tf.math.log(output)
        hce += (1 - mask) * tf.math.pow((1 - target), self.beta) * tf.math.pow(output, self.alpha) * tf.math.log(1 - output)
        
        if self.reduce:
            hce = tf.math.reduce_mean(hce)
        
        return -hce

In [51]:
model.compile(
    optimizer=optimizers.SGD(learning_rate=2.),
    loss=[HeatMapCrossEntropy(), losses.Huber(), losses.Huber()],
    metrics=[metrics.MeanSquaredError(), metrics.MeanSquaredError(), metrics.MeanSquaredError()]
)

In [5]:
bce = losses.BinaryCrossentropy(reduction=None)
bce(tf.constant([1, 0, 0]), tf.constant([0.1, 0.8, 0.1])).numpy()

1.3391279

In [6]:
hce = HeatMapCrossEntropy()
hce(tf.constant([1, 0, 0.9]), tf.constant([0.1, 0.8, 0.1])).numpy()

0.96504474

In [7]:
input_dir = "../data/"
skeleton_loc = input_dir + "0002-L_Skeleton.txt"
label_loc = input_dir + "0002-L_Label.txt"

width = 2048
label_count = 43

In [4]:
sk_image, sk_motion, frames = tad_data.get_skeleton(skeleton_loc, width)

In [23]:
heatmap_t, offset_t, size_t = tad_data.get_target(label_loc, label_count, frames, width)

In [26]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

print(size_t[:,0,0])

np.set_printoptions(**opt)

tf.Tensor(
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         

In [21]:
model((tf.expand_dims(sk_image, axis=0), tf.expand_dims(sk_motion, axis=0)))

[<tf.Tensor: shape=(1, 1024, 1, 10), dtype=float32, numpy=
 array([[[[0.4997304 , 0.4998675 , 0.5009979 , ..., 0.50067747,
           0.5008321 , 0.5000075 ]],
 
         [[0.4997842 , 0.5005114 , 0.49996978, ..., 0.5002949 ,
           0.4999284 , 0.4995913 ]],
 
         [[0.49933395, 0.4998647 , 0.5007191 , ..., 0.50043434,
           0.5011411 , 0.5006604 ]],
 
         ...,
 
         [[0.50009364, 0.5009308 , 0.49932548, ..., 0.4996067 ,
           0.50085294, 0.501408  ]],
 
         [[0.50061953, 0.5010755 , 0.5004234 , ..., 0.5000047 ,
           0.500351  , 0.50181615]],
 
         [[0.50018126, 0.5004938 , 0.49986374, ..., 0.4998187 ,
           0.50012445, 0.500413  ]]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1024, 1, 1), dtype=float32, numpy=
 array([[[[ 0.00062085]],
 
         [[-0.0019378 ]],
 
         [[-0.00036911]],
 
         ...,
 
         [[-0.00292208]],
 
         [[-0.00750423]],
 
         [[-0.00361626]]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1024, 1,

In [28]:
model.evaluate((tf.expand_dims(sk_image, axis=0), tf.expand_dims(sk_motion, axis=0)), (tf.expand_dims(heatmap_t, axis=0), tf.expand_dims(offset_t, axis=0), tf.expand_dims(size_t, axis=0)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - conv2d_10_mean_squared_error: 5.0859e-04 - conv2d_12_mean_squared_error: 0.0304 - conv2d_8_mean_squared_error: 0.2492 - loss: 0.1815


[0.18151390552520752,
 0.0005085895536467433,
 0.030351724475622177,
 0.24923500418663025]